In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re
import sys
import os
import dotenv 
dotenv.load_dotenv()
from datetime import datetime

key = os.getenv("token")

In [13]:
##sacar las skid
parametro = 'ovd'
url = "https://sky-scrapper.p.rapidapi.com/api/v1/flights/searchAirport"

querystring = {"query":f"{parametro}"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': True, 'timestamp': 1729412962988, 'data': [{'skyId': 'OVD', 'entityId': '95565102', 'presentation': {'title': 'Asturias', 'suggestionTitle': 'Asturias (OVD)', 'subtitle': 'Spain'}, 'navigation': {'entityId': '95565102', 'entityType': 'AIRPORT', 'localizedName': 'Asturias', 'relevantFlightParams': {'skyId': 'OVD', 'entityId': '95565102', 'flightPlaceType': 'AIRPORT', 'localizedName': 'Asturias'}, 'relevantHotelParams': {'entityId': '46362301', 'entityType': 'CITY', 'localizedName': 'Santiago del Monte'}}}]}


In [ ]:
#  skyId': 'MAD', 'entityId': '95565077'
# 'skyId': 'AGP', 'entityId': '95565095'
# 'skyId': 'OVD', 'entityId': '95565102'

In [2]:


url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlightsComplete"

querystring = {"originSkyId":"MAD","destinationSkyId":"AGP","originEntityId":"95565077","destinationEntityId":"95565095","date":"2024-11-01","returnDate":"2024-11-03","adults":"2","currency":"EUR"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

if response.status_code == 200:
    data = response.json()

{'status': False, 'timestamp': 1729420502913, 'message': 'Something went wrong. We have logged the error and will get the resolution.'}


In [ ]:
#corrigiendolo ha petado la api

In [ ]:
res= data["data"]
precio= res["itineraries"][0]["price"]["raw"]

origen= res["itineraries"][0]["legs"][0]["origin"]["city"]

destino= res["itineraries"][0]["legs"][0]["destination"]["city"]

duracion_minuts= res["itineraries"][0]["legs"][0]["durationInMinutes"]

salida= res["itineraries"][0]["legs"][0]["departure"]
llegada= res["itineraries"][0]["legs"][0]["arrival"]

aerolinea= res["itineraries"][0]["legs"][0]["carriers"]["marketing"][0]["name"]

In [16]:
dicct= {
       "ciudad_origen":origen,
       "ciudad_destino":destino,
       "salida":salida,
       "llegada":llegada,
       "duracion(min)":duracion_minuts,
       "aerolinea":aerolinea,
       "precio":precio
        }
dicct

{'ciudad_origen': 'Madrid',
 'ciudad_destino': 'Málaga',
 'salida': '2024-11-01T10:30:00',
 'llegada': '2024-11-01T11:40:00',
 'duracion(min)': 70,
 'aerolinea': 'Air Europa',
 'precio': 352.44}

In [17]:
lista_dicct=[]

for vuelo in range(0,len(res["itineraries"])):
        res= data["data"]
        precio= res["itineraries"][0]["price"]["raw"]
        for i in range(0,2):
                origen= res["itineraries"][vuelo]["legs"][i]["origin"]["city"]
                destino= res["itineraries"][vuelo]["legs"][i]["destination"]["city"]

                duracion_minuts= res["itineraries"][vuelo]["legs"][i]["durationInMinutes"]

                salida= res["itineraries"][vuelo]["legs"][i]["departure"]
                llegada= res["itineraries"][vuelo]["legs"][i]["arrival"]

                aerolinea= res["itineraries"][vuelo]["legs"][i]["carriers"]["marketing"][0]["name"]

                dicct= {
                "ciudad_origen":origen,
                "ciudad_destino":destino,
                "salida":salida,
                "llegada":llegada,
                "duracion(min)":duracion_minuts,
                "aerolinea":aerolinea,
                "precio":precio
                }
                lista_dicct.append(dicct)
df_vuelos_malaga= pd.DataFrame(lista_dicct)
df_vuelos_malaga.head(2)

,ciudad_origen,ciudad_destino,salida,llegada,duracion(min),aerolinea,precio
0,Madrid,Málaga,2024-11-01T10:30:00,2024-11-01T11:40:00,70,Air Europa,352.44
1,Málaga,Madrid,2024-11-03T20:50:00,2024-11-03T22:05:00,75,Air Europa,352.44


In [18]:
df_vuelos_malaga.isnull().sum()



ciudad_origen     0
ciudad_destino    0
salida            0
llegada           0
duracion(min)     0
aerolinea         0
precio            0
dtype: int64

In [19]:
df_vuelos_malaga.to_csv('Datos/vuelos_malaga.csv')

In [3]:
df_vuelos = pd.read_csv('../Datos/vuelos_malaga.csv')
df_vuelos.head(2)

,Unnamed: 0,ciudad_origen,ciudad_destino,salida,llegada,duracion(min),aerolinea,precio
0,0,Madrid,Málaga,2024-11-01T10:30:00,2024-11-01T11:40:00,70,Air Europa,352.44
1,1,Málaga,Madrid,2024-11-03T20:50:00,2024-11-03T22:05:00,75,Air Europa,352.44


In [4]:
df_vuelos.dtypes

Unnamed: 0          int64
ciudad_origen      object
ciudad_destino     object
salida             object
llegada            object
duracion(min)       int64
aerolinea          object
precio            float64
dtype: object

In [5]:
df_vuelos.drop(columns=['Unnamed: 0'], inplace=True)
df_vuelos[['fecha_salida', 'hora_salida']] = df_vuelos['salida'].str.split('T', expand=True)
df_vuelos[['fecha_llegada', 'hora_llegada']] = df_vuelos['llegada'].str.split('T', expand=True)
df_vuelos.dtypes

ciudad_origen      object
ciudad_destino     object
salida             object
llegada            object
duracion(min)       int64
aerolinea          object
precio            float64
fecha_salida       object
hora_salida        object
fecha_llegada      object
hora_llegada       object
dtype: object

In [6]:
df_vuelos['fecha_salida'] = pd.to_datetime(df_vuelos['fecha_salida'])
df_vuelos['hora_salida'] = pd.to_datetime(df_vuelos['hora_salida'])
df_vuelos['fecha_llegada'] = pd.to_datetime(df_vuelos['fecha_llegada'])
df_vuelos['hora_llegada'] = pd.to_datetime(df_vuelos['hora_llegada'])

C:\Users\HP\AppData\Local\Temp\ipykernel_9520\161310273.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos['hora_salida'] = pd.to_datetime(df_vuelos['hora_salida'])
C:\Users\HP\AppData\Local\Temp\ipykernel_9520\161310273.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos['hora_llegada'] = pd.to_datetime(df_vuelos['hora_llegada'])


In [7]:
df_vuelos.drop(columns=['salida', 'llegada'], inplace=True)

In [8]:
df_vuelos.head()

,ciudad_origen,ciudad_destino,duracion(min),aerolinea,precio,fecha_salida,hora_salida,fecha_llegada,hora_llegada
0,Madrid,Málaga,70,Air Europa,352.44,2024-11-01,2024-10-20 10:30:00,2024-11-01,2024-10-20 11:40:00
1,Málaga,Madrid,75,Air Europa,352.44,2024-11-03,2024-10-20 20:50:00,2024-11-03,2024-10-20 22:05:00
2,Madrid,Málaga,70,Air Europa,352.44,2024-11-01,2024-10-20 10:30:00,2024-11-01,2024-10-20 11:40:00
3,Málaga,Madrid,75,Air Europa,352.44,2024-11-03,2024-10-20 08:30:00,2024-11-03,2024-10-20 09:45:00
4,Madrid,Málaga,70,Air Europa,352.44,2024-11-01,2024-10-20 19:00:00,2024-11-01,2024-10-20 20:10:00


In [10]:
df_vuelos.to_csv('../Datos/vuelos_malaga.csv')